# Google Trends data cleaning

In [136]:
import pandas as pd
import numpy as np
import csv

In [137]:
df_yoo=pd.DataFrame({(2,2),(11,2)},{1,2})
df_yoo

,0,1
1,11,2
2,2,2


In [138]:
# Getting the code
df_google = pd.DataFrame()
file_names = ['Gtrend_data_untidy/Gtrend_protest_data.csv', 'Gtrend_data_untidy/Gtrend_revolution_data.csv', 'Gtrend_data_untidy/Gtrend_riots_data.csv', 'Gtrend_data_untidy/Gtrend_strike_data.csv', 'Gtrend_data_untidy/Gtrend_violence_data.csv', 'Gtrend_data_untidy/Gtrend_unrest_data.csv']
for file_name in file_names:
    with open(file_name, 'r') as file:
        next(file)
        data = csv.reader(file)
        df0 = pd.DataFrame(data)
        df_google = pd.concat([df_google, df0])

# Reading matching data
with open("country_codes.csv", 'r') as file:
    next(file)
    data = csv.reader(file)
    countries_df = pd.DataFrame(data)
countries_df.columns = column_names = ["country", "geo", "country_code", "number"]
countries_df = countries_df.drop(columns="number")

df_google

,0,1,2,3,4,5,6,7
0,1,2004-01-01,0,protest,AF,all,web,0
1,2,2004-02-01,0,protest,AF,all,web,0
2,3,2004-03-01,100,protest,AF,all,web,0
3,4,2004-04-01,0,protest,AF,all,web,0
4,5,2004-05-01,0,protest,AF,all,web,0
...,...,...,...,...,...,...,...,...
28285,28286,2022-10-01,0,unrest,ZW,all,web,0
28286,28287,2022-11-01,<1,unrest,ZW,all,web,0
28287,28288,2022-12-01,0,unrest,ZW,all,web,0
28288,28289,2023-01-01,0,unrest,ZW,all,web,0


In [139]:
# labelling and dropping columns
df_google.columns = column_names = ["code", "date", "hits", "keyword", "geo", "time", "gprop", "category"]
df_google = df_google.drop(columns=["code", "time", "gprop", "category"])
df_google = df_google.replace(to_replace="<1", value=0.5) # replace "<1" values with "0.5"

# day month year
df_google["date"] = pd.to_datetime(df_google["date"])
df_google["day"] = df_google["date"].dt.day
df_google["month"] = df_google["date"].dt.month
df_google["year"] = df_google["date"].dt.year
df_google = df_google.drop(columns=["day"])

# matching on country codes
merged_df_google = pd.DataFrame.merge(df_google, countries_df)
merged_df_google

# selecting only the years we are interested in; i.e. pre-2019
merged_df_google = merged_df_google[merged_df_google["year"] <= 2019]

merged_df_google.to_csv(r'google_all_data_monthly.csv')
merged_df_google

,date,hits,keyword,geo,month,year,country,country_code
0,2004-01-01,0,protest,AF,1,2004,Afghanistan,AFG
1,2004-02-01,0,protest,AF,2,2004,Afghanistan,AFG
2,2004-03-01,100,protest,AF,3,2004,Afghanistan,AFG
3,2004-04-01,0,protest,AF,4,2004,Afghanistan,AFG
4,2004-05-01,0,protest,AF,5,2004,Afghanistan,AFG
...,...,...,...,...,...,...,...,...
253417,2019-08-01,0,violence,KI,8,2019,Kiribati,KIR
253418,2019-09-01,5,violence,KI,9,2019,Kiribati,KIR
253419,2019-10-01,0,violence,KI,10,2019,Kiribati,KIR
253420,2019-11-01,0,violence,KI,11,2019,Kiribati,KIR


In [140]:
# taking yearly average
merged_df_google["date"] = pd.to_datetime(merged_df_google["date"])
merged_df_google["hits"] = pd.to_numeric(merged_df_google["hits"], errors='coerce') # non-numerical values converted to NaN
merged_df_google.set_index("date", inplace=True) # set date column as index
merged_df_google = merged_df_google.groupby("country_code")
df_google_yearly = merged_df_google["hits"].resample("Y").mean() #resampling
df_google_yearly.to_csv(r'google_all_data_yearly.csv')
df_google_yearly

country_code  date      
ABW           2004-12-31    10.116667
              2005-12-31     8.833333
              2006-12-31     5.983333
              2007-12-31     4.066667
              2008-12-31     1.466667
                              ...    
ZWE           2015-12-31     3.159722
              2016-12-31     4.375000
              2017-12-31     3.111111
              2018-12-31     3.402778
              2019-12-31     3.423611
Name: hits, Length: 3568, dtype: float64

In [141]:
print(merged_df_google["hits"].isnull().sum())

AttributeError: 'SeriesGroupBy' object has no attribute 'isnull'